In [43]:
import cv2
import mediapipe as mp
import numpy as np
import math
import os
import matplotlib.pyplot as plt

In [44]:
mp_hands = mp.solutions.hands

In [45]:
def get_hand_image(image):
    if(image is not None):
        width = image.shape[1]
        height = image.shape[0]
        with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = hands.process(image)
            image.flags.writeable = True
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    landmarks = np.array([[lm.x * width, lm.y * height] for lm in hand_landmarks.landmark])
                    hand_bbox = cv2.boundingRect(landmarks.astype(np.float32))

                    # Increase the size of the bounding box by 100 pixels in all directions
                    hand_bbox = (hand_bbox[0] - 100, hand_bbox[1] - 100, hand_bbox[2] + 200, hand_bbox[3] + 200)
                    # Check if the hand bounding box is valid
                    if hand_bbox[2] > 0 and hand_bbox[3] > 0:
                        # Extract hand image
                        hand_image = image[hand_bbox[1]:hand_bbox[1] + hand_bbox[3], hand_bbox[0]:hand_bbox[0] + hand_bbox[2]]
                        # Check if hand_image is not empty
                        if hand_image.size != 0:
                            # Resize hand image
                            hand_image = cv2.resize(hand_image, (64, 64))
                            return hand_image
                        else:
                            print("Error: Empty hand image")
                    else:
                        print("Error: Invalid hand bounding box")
            else:
                print("Error: No hand detected")
        # Return None if no hand image is found
        return None


In [46]:
def create_seq_of_frames(vidPath):
    sequence = []
    seq_len = 30
    
    vid = cv2.VideoCapture(vidPath)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    print("There are {} frames to be processed ".format(total_frames))
    buffer = np.zeros((64, 64,3))
    # Determine the step size to select frames evenly
    step = max(total_frames // seq_len, 1)
    for frame_num in range(0, total_frames, step):
        vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        _, frame = vid.read()

        hand_frame = get_hand_image(frame)
        # plt.imshow(hand_frame,cmap='gray')

        if hand_frame is None:
            sequence.append(buffer/255)
        else:
            sequence.append(hand_frame / 255)
            buffer = hand_frame
    # Apply pre-padding if necessary
    while len(sequence) < seq_len:
        sequence.append(buffer / 255)
    
    vid.release()
    return sequence[:seq_len] 

In [47]:
def create_dataset(parentFolder,dataset):
    for file in os.listdir(parentFolder):
        filePath = "{}/{}".format(parentFolder,file)
        
        print("Processing " + filePath)
        dataset.append(create_seq_of_frames(filePath))
    print(np.array(dataset).shape)
    return dataset

In [48]:
x = []
y = []

In [49]:
x = create_dataset("Dataset2/Correct",x)
x = create_dataset("Dataset2/Incorrect",x)

Processing Dataset2/Correct/20240330_184451.mp4
There are 47 frames to be processed 
Error: Empty hand image
Error: Empty hand image
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: Empty hand image
Error: Empty hand image
Processing Dataset2/Correct/20240330_184455.mp4
There are 45 frames to be processed 
Error: Empty hand image
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: Empty hand image
Processing Dataset2/Correct/20240330_184458.mp4
There are 56 frames to be processed 
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: No hand detected
Error: Empty hand image
P

In [50]:
y = [1]*102 + [0]*106

In [51]:
import pickle

In [52]:
dataset = {
    "features":x,
    "labels":y
}

In [53]:
with open ("30frameDatasetLargeRect.pkl","wb") as f:
    pickle.dump(dataset,f)